In [1]:
import sys
import findspark
import pyspark
import folium
import pandas as pd
findspark.init("c:/Users/Eigenaar/spark-3.1.2-bin-hadoop3.2")

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

In [3]:
#Creating schema's
schema_pubs = StructType([
    StructField("Food Standard Agency's ID", StringType(), nullable=True),
    StructField("name", StringType(), nullable=True),
    StructField("address", StringType(), nullable=True),
    StructField("postcode", StringType(), nullable=True),
    StructField("easting", DoubleType(), nullable=True),
    StructField("northing", DoubleType(), nullable=True),
    StructField("latitude", DoubleType(), nullable=False),
    StructField("longitude", DoubleType(), nullable=False),
    StructField("local_authority", StringType(), nullable=True)

])

In [4]:
# importing data 
accidentData_df = spark.read.format('csv').option('header',True).load('Accidents0514.csv')
pubsData_df = spark.read.format('csv').option('header',False).schema(schema_pubs).load('open_pubs.csv')

In [5]:
## Toon de pub data set
## Somige rows van latitude en longitude bevate "none" waarden waardoor we hie weinig mee konden doen, deze filteren we er hier uit
counterRowsBeforePubs=pubsData_df.count()
pubsData_df=pubsData_df.dropna(subset=["latitude","longitude"])


In [9]:
print(counterRowsBeforePubs-pubsData_df.count()," rijen zijn verwijderd omdat ze lege waarden hadden.")


146  rijen zijn verwijderd omdat ze lege waarden hadden


In [ ]:
## Toon de accident data set
accidentData_df.show()


In [ ]:
#Trying to create a map with pubs
UK_COORDINATES = (53.5500, -6.4333)
map = folium.Map(location=UK_COORDINATES, zoom_start=5.5)
## testing 
print(map)
counter=0
for row in pubsData_df.rdd.collect():
    folium.Marker(
        [row.latitude, row.longitude], popup=row.name, tooltip=row.name
     ).add_to(map)
    if type(row.latitude)==None:
        print(row)
        break
display(map)